In [ ]:
"from lightglue import LightGlue, SuperPoint, viz2d, DISK, SIFT, ALIKED, DoGHardNet\n",
    "from lightglue.utils import load_image, rbd, match_pair\n",
    "import matplotlib.pyplot as plt\n",
    "import cv2 as cv\n",
    "import numpy as np\n",
    "import torch\n",
    "import torchvision.transforms.functional as TF\n",
    "import os\n",
    "import cv2\n",
    "import pandas as pd  # Import pandas\n",
    "import glob\n",
    "import math\n",
    "from tqdm.notebook import tqdm\n",
    "\n",
    "general_folder_path = '/home/oussama/Documents/EPFL/PDS_LUTS/'\n",
    "\n",
    "output_path = general_folder_path + 'panorama.jpg'\n",
    "\n",
    "device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\n",
    "\n",
    "torch.cuda.empty_cache()\n",
    "\n",
    "# Utility function to calculate image corners in homogeneous coordinates\n",
    "def get_homogeneous_corners(width, height):\n",
    "    return np.array([[0, 0, 1], [width, 0, 1], [width, height, 1], [0, height, 1]]).T\n",
    "\n",
    "def warp_perspective_padded1(src, dst, transf):\n",
    "    src_h, src_w = src.shape[:2]\n",
    "    dst_h, dst_w = dst.shape[:2]\n",
    "\n",
    "    # Define the corners of the src image\n",
    "    src_corners = np.array([\n",
    "        [0, 0],\n",
    "        [src_w, 0],\n",
    "        [src_w, src_h],\n",
    "        [0, src_h]\n",
    "    ], dtype=np.float32)\n",
    "\n",
    "    # Transform the src corners using the homography matrix (transf)\n",
    "    src_corners_transformed = cv2.perspectiveTransform(src_corners[None, :, :], transf)[0]\n",
    "\n",
    "    # Define the corners of the dst image in its own coordinate space\n",
    "    dst_corners = np.array([\n",
    "        [0, 0],\n",
    "        [dst_w, 0],\n",
    "        [dst_w, dst_h],\n",
    "        [0, dst_h]\n",
    "    ], dtype=np.float32)\n",
    "\n",
    "    # Combine all corners to find the overall bounding box\n",
    "    all_corners = np.vstack((src_corners_transformed, dst_corners))\n",
    "\n",
    "    # Compute the bounding box of all corners\n",
    "    x_min, y_min = np.int32(all_corners.min(axis=0))\n",
    "    x_max, y_max = np.int32(all_corners.max(axis=0))\n",
    "\n",
    "    # Calculate the translation needed to shift images to positive coordinates\n",
    "    shift_x = -x_min\n",
    "    shift_y = -y_min\n",
    "\n",
    "    # Compute the size of the output canvas\n",
    "    output_width = x_max - x_min\n",
    "    output_height = y_max - y_min\n",
    "\n",
    "    # Compute the 3x3 translation matrix to shift the images\n",
    "    translation_matrix = np.array([\n",
    "        [1, 0, shift_x],\n",
    "        [0, 1, shift_y],\n",
    "        [0, 0,      1]\n",
    "    ], dtype=np.float32)\n",
    "\n",
    "    # Update the transformation matrix to include the translation\n",
    "    new_transf = translation_matrix @ transf\n",
    "\n",
    "    # Warp the src image using the updated transformation matrix\n",
    "    warped = cv2.warpPerspective(src, new_transf, (output_width, output_height))\n",
    "    \n",
    "    # Warp the dst image using only the translation matrix (affine)\n",
    "    dst_pad = cv2.warpAffine(dst, translation_matrix[:2], (output_width, output_height))\n",
    "\n",
    "    # Determine the anchor points\n",
    "    anchorX = int(shift_x)\n",
    "    anchorY = int(shift_y)\n",
    "\n",
    "    # Determine the sign\n",
    "    sign = (anchorX > 0) or (anchorY > 0)\n",
    "\n",
    "    return dst_pad, warped, anchorX, anchorY, sign\n",
    "\n",
    "# Rotate image tensor by specified angle\n",
    "def rotate_image(image, angle):\n",
    "    return TF.rotate(image, angle)\n",
    "\n",
    "def rotate_image1(image, angle):\n",
    "    \"\"\"\n",
    "    Rotate a PyTorch image tensor without cropping and add necessary padding to preserve all content.\n",
    "    Keeps the tensor on its original device (CPU or CUDA).\n",
    "    Args:\n",
    "        image: A PyTorch tensor in CHW format with values in [0, 1].\n",
    "        angle: Rotation angle in degrees (counterclockwise).\n",
    "    Returns:\n",
    "        Rotated image as a PyTorch tensor on the same device.\n",
    "    \"\"\"\n",
    "    device = image.device  # Store original device (CPU or CUDA)\n",
    "\n",
    "    # Convert tensor to NumPy array (HWC format)\n",
    "    if isinstance(image, torch.Tensor):\n",
    "        image_np = image.permute(1, 2, 0).cpu().numpy()  # CHW -> HWC on CPU\n",
    "        if image_np.max() <= 1:  # Scale up to [0, 255] if needed\n",
    "            image_np = (image_np * 255).astype(np.uint8)\n",
    "    else:\n",
    "        raise TypeError(\"Input must be a PyTorch tensor in CHW format.\")\n",
    "\n",
    "    # Get the height and width of the image\n",
    "    h, w = image_np.shape[:2]\n",
    "    center = (w // 2, h // 2)\n",
    "\n",
    "    # Compute the rotation matrix and new dimensions\n",
    "    rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)\n",
    "    cos_val = abs(rotation_matrix[0, 0])\n",
    "    sin_val = abs(rotation_matrix[0, 1])\n",
    "    new_w = int((h * sin_val) + (w * cos_val))\n",
    "    new_h = int((h * cos_val) + (w * sin_val))\n",
    "\n",
    "    # Adjust the rotation matrix to account for translation\n",
    "    rotation_matrix[0, 2] += (new_w / 2) - center[0]\n",
    "    rotation_matrix[1, 2] += (new_h / 2) - center[1]\n",
    "\n",
    "    # Perform rotation with padding\n",
    "    rotated_image_np = cv.warpAffine(\n",
    "        image_np, rotation_matrix, (new_w, new_h),\n",
    "        flags=cv.INTER_CUBIC,\n",
    "        borderMode=cv.BORDER_CONSTANT,\n",
    "        borderValue=(0, 0, 0)\n",
    "    )\n",
    "\n",
    "    # Convert back to PyTorch tensor (CHW format)\n",
    "    rotated_image_tensor = torch.from_numpy(rotated_image_np).permute(2, 0, 1).float()\n",
    "    if rotated_image_tensor.max() > 1:  # Normalize back to [0, 1]\n",
    "        rotated_image_tensor /= 255.0\n",
    "\n",
    "    # Move the tensor back to the original device\n",
    "    return rotated_image_tensor.to(device)\n",
    "\n",
    "# Calculate similarity score between two images\n",
    "def compute_similarity_score(image1, image2, extractor, matcher):\n",
    "    with torch.no_grad():\n",
    "        feats0 = extractor.extract(image1)\n",
    "        feats1 = extractor.extract(image2)\n",
    "        feats0, feats1, matches01 = match_pair(extractor, matcher, image1, image2)\n",
    "    points0 = feats0['keypoints'][matches01['matches'][..., 0]]\n",
    "    score = points0.shape[0]\n",
    "    # Release GPU memory\n",
    "    del feats0, feats1, matches01, points0\n",
    "    torch.cuda.empty_cache()\n",
    "    return score\n",
    "\n",
    "def split_image_paths(image_paths):\n",
    "    \"\"\"\n",
    "    Splits the image_paths list into two halves:\n",
    "    - First half gets an extra frame if the total number of images is odd.\n",
    "    - Second half is reversed to start from the last image and go to the middle.\n",
    "\n",
    "    Args:\n",
    "        image_paths (list): List of image paths.\n",
    "\n",
    "    Returns:\n",
    "        tuple: (first_half, second_half) - two lists of image paths.\n",
    "    \"\"\"\n",
    "    # Compute the midpoint\n",
    "    mid = (len(image_paths) + 1) // 2\n",
    "\n",
    "    # Split the list\n",
    "    first_half = image_paths[:mid]  # First half\n",
    "    second_half = image_paths[mid:][::-1]  # Second half, reversed\n",
    "\n",
    "    return first_half, second_half\n",
    "\n",
    "# Feature extractor and matcher initialization\n",
    "extractor = DoGHardNet(max_num_keypoints=None).eval().cuda()\n",
    "matcher = LightGlue(features='doghardnet').eval().cuda()\n",
    "\n",
    "start = True\n"